In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv2D, MaxPooling2D, Flatten, MaxPooling1D, Conv1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df=pd.read_csv('input9.csv')

In [3]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [4]:
df['class'].value_counts()

WITHOUT_CLASSIFICATION    58204
DESIGN                     2703
IMPLEMENTATION              757
DEFECT                      472
TEST                         85
DOCUMENTATION                54
Name: class, dtype: int64

In [5]:
df['class']=df['class'].apply(lambda x: 0 if x=='WITHOUT_CLASSIFICATION' else 1)

In [6]:
df['class'].value_counts()

0    58204
1     4071
Name: class, dtype: int64

In [7]:
X = df['comment']
Y = df['class']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify=Y, random_state=42)

max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
train_sequences = tok.texts_to_sequences(X_train)
train_sequences_matrix = pad_sequences(train_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [9]:
print(len(train_sequences_matrix[0]))

250


In [10]:
def CNN():
    cnn = tf.keras.Sequential()
    cnn.add(Input(name='inputs',shape=[max_len]))
    cnn.add(Embedding(max_words,50,input_length=max_len))
    cnn.add(Conv1D(128, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(256, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))
    cnn.add(Conv1D(512, kernel_size=3, activation='relu', padding="same"))
    cnn.add(MaxPooling1D(pool_size=2))

    cnn.add(Flatten())
    cnn.add(Dense(256, activation ='relu'))
    cnn.add(Dense(64, activation ='relu'))
    cnn.add(Dense(1, activation='sigmoid'))

    return cnn

In [14]:
model = CNN()
model.summary()
opt = tf.keras.optimizers.Adadelta(learning_rate=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 50)           500000    
                                                                 
 conv1d_3 (Conv1D)           (None, 250, 128)          19328     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 125, 128)         0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 125, 256)          98560     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 62, 256)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 62, 512)          

In [15]:
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(train_sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.25, callbacks=[model_checkpoint_callback,reduce_lr])

model.load_weights(checkpoint_filepath)

Epoch 1/50
292/292 [==============================] - 8s 26ms/step - loss: 0.2640 - accuracy: 0.9309 - val_loss: 0.2232 - val_accuracy: 0.9375 - lr: 0.1000
Epoch 2/50
292/292 [==============================] - 7s 23ms/step - loss: 0.2246 - accuracy: 0.9337 - val_loss: 0.2068 - val_accuracy: 0.9375 - lr: 0.1000
Epoch 3/50
292/292 [==============================] - 7s 23ms/step - loss: 0.2156 - accuracy: 0.9337 - val_loss: 0.2054 - val_accuracy: 0.9375 - lr: 0.1000
Epoch 4/50
292/292 [==============================] - 7s 23ms/step - loss: 0.2076 - accuracy: 0.9336 - val_loss: 0.1941 - val_accuracy: 0.9373 - lr: 0.1000
Epoch 5/50
292/292 [==============================] - 7s 23ms/step - loss: 0.1782 - accuracy: 0.9370 - val_loss: 0.1284 - val_accuracy: 0.9578 - lr: 0.1000
Epoch 6/50
292/292 [==============================] - 7s 23ms/step - loss: 0.1151 - accuracy: 0.9650 - val_loss: 0.0986 - val_accuracy: 0.9724 - lr: 0.1000
Epoch 7/50
292/292 [==============================] - 7s 23ms/st

In [16]:
model.save("CNN_imbalanced.h5")

In [17]:
accr = model.evaluate(test_sequences_matrix,Y_test)

390/390 [==============================] - 2s 5ms/step - loss: 0.0721 - accuracy: 0.9777


In [18]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.072
  Accuracy: 0.978


In [19]:
y_pred = model.predict(test_sequences_matrix)

390/390 [==============================] - 2s 4ms/step


In [20]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [21]:
print(classification_report(Y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.988479  0.987630  0.988054     11641
           1   0.825243  0.835381  0.830281       814

    accuracy                       0.977680     12455
   macro avg   0.906861  0.911505  0.909168     12455
weighted avg   0.977811  0.977680  0.977743     12455

